In [1]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

%matplotlib inline

In [2]:
pd.set_option('display.max_colwidth', 280)
train_neg = pd.read_csv("../input/arabic-sentiment-twitter-corpus/train_Arabic_tweets_negative_20190413.tsv", sep="\t", header=None,  quoting=csv.QUOTE_NONE)
train_neg.rename(columns={0:'label', 1:'tweet'}, inplace=True)
train_neg['label'] = 0

train_pos = pd.read_csv("../input/arabic-sentiment-twitter-corpus/train_Arabic_tweets_positive_20190413.tsv", sep="\t", header=None,  quoting=csv.QUOTE_NONE)
train_pos.rename(columns={0:'label', 1:'tweet'}, inplace=True)
train_pos['label'] = 1


train_df = pd.concat([train_neg, train_pos], axis=0).reset_index(drop=True)


from sklearn.model_selection import train_test_split
X = train_df.tweet.values
y = train_df.label.values

# The train val split is used by the DL approach but not classical ML
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.1, random_state=2020)
# Load test subset
test_pos = pd.read_csv("../input/arabic-sentiment-twitter-corpus/test_Arabic_tweets_positive_20190413.tsv", sep="\t", header=None,  quoting=csv.QUOTE_NONE)
test_pos.rename(columns={0:'label', 1:'tweet'}, inplace=True)
test_pos['label']=1

test_neg = pd.read_csv("../input/arabic-sentiment-twitter-corpus/test_Arabic_tweets_negative_20190413.tsv", sep="\t", header=None,  quoting=csv.QUOTE_NONE)
test_neg.rename(columns={0:'label', 1:'tweet'}, inplace=True)
test_neg['label']=0

test_df = pd.concat([test_neg, test_pos], axis=0).reset_index(drop=True)
X_test = test_df.tweet.values
y_test = test_df.label.values

In [3]:
train_df.head()

,label,tweet
0,0,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم بالزايد 😭
1,0,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين احس فيه احد ناقصهم 💔 #Avlu
2,0,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال والاهلي تحت التاق 👇 #تحدي_اسرع_روقان وادخل في سحب قيمة ايفون X على…
3,0,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين على بكتيريا 🦠 فتنفجر 💥 و تموت . الأخيرة يبدو انها بكتيريا مقاومة فأخذ…
4,0,الدودو جايه تكمل علي 💔


In [4]:
train_df.shape

(47000, 2)

In [5]:
df_ss2030 = pd.read_csv("../input/arabic-sentiment-analysis-dataset-ss2030-dataset/Arabic Sentiment Analysis Dataset - SS2030.csv")
# Rename columns to match convention
df_ss2030 = df_ss2030.rename(columns = {"text":"tweet", "Sentiment": "label"})

In [6]:
df_ss2030.head()

,tweet,label
0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,1
1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. https://t.co/ps3qNw1CbB,1
2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (@Shubratanmyeh):\n \n ما زال التسجيل مستمر في دورة حقوق المرأة بعد الطلاق ✨ #وعيك_يحميك... https://t.co/c2NXzNCdLU,1
3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة العدل https://t.co/QUGzWwubFk,1
4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او ولي المراة من الاخطاء الشائعة \n \n هذا الكلام غلط في الشريعة والقانون\n فلا يوجد ولي للزوجة او المراة الا اذا كانت قاصرا ويكون الولي ابوها ...الخ وليس الزوج منهم\n نعم له حقوق عليها لكنها ليست ولاية\n الولاية مصطلح فقهي قانوني ...,1


In [7]:
df_ss2030.shape

(4252, 2)

In [8]:
df_reviews = pd.read_csv("../input/arabic-100k-reviews/ar_reviews_100k.tsv", delimiter="\t")
# Create a mapping for the labels such that we use the same convention across all datasets
label_mapping = {"Positive": 1, "Negative":0}
# Filter to only have pos and neg tweets, i.e: remove mixed tweets
df_reviews = df_reviews[df_reviews.label != "Mixed"]
df_reviews["label"] = df_reviews["label"].map(label_mapping)
# Rename columns to match convention
df_reviews = df_reviews.rename(columns = {"text":"tweet"})

In [9]:
df_reviews.head()

,label,tweet
0,1,ممتاز نوعا ما . النظافة والموقع والتجهيز والشاطيء. المطعم
1,1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدولة يعشق ترابها. نحن نحب الإمارات. ومضات من فكر. نصائح لدولة تطمح بالصفوف الأولى و قائد لا يقبل إلا براحة شعبه وتوفر كل سب العيش الكريم. حكم و مواقف ونصائح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد. يستحق القراءة مرات كثيرة
2,1,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى هدوء جبال الشيشان .. للتعرف على حقيقة ما يجرى فى تلك البلاد من حروب ضاربة بحق المسلمين و جزء كبير من تاريخ تلك المنطقة. التضحية .. الرجولة .. الوفاء والكثير من القيم الأخرى اثبتت وجودها فى تلك الرواية البسيطة
3,1,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن.. احمد مراد تخطى مرحلة ان القارئ يخلص الرواية وهو فاتح بؤه لمرحلة ان القارئ يخلص الرواية وهو محترم الكاتب.. اتقان مخيف.. بصرف النظر عن اخطاء لا تذكر ف الحوار.. انما احمد مراد سافر عاش حبة ف اوائل القرن العشرين وجه ي...
4,1,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل الخدمات مريح نفسيا. لا يوجد


In [10]:
df_reviews.shape

(66666, 2)

In [11]:
# Helper functions 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
def train_model(model, data, targets):
    text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', model),
    ])
    text_clf.fit(data, targets)
    return text_clf
def get_accuracy(trained_model,X, y):
    predicted = trained_model.predict(X)
    accuracy = np.mean(predicted == y)
    return accuracy

In [12]:
from sklearn.naive_bayes import MultinomialNB
trained_clf_multinomial_nb = train_model(MultinomialNB(), X, y)
accuracy = get_accuracy(trained_clf_multinomial_nb,X_test, y_test)
print(f"Test dataset accuracy with MultinomialNB: {accuracy:.2f}")

Test dataset accuracy with MultinomialNB: 0.79


In [13]:
def print_all_accuracies(dataset_name, dataset):
  accuracy = get_accuracy(trained_clf_multinomial_nb,dataset.tweet.values, dataset.label.values)
  print(f"{dataset_name} dataset accuracy with Multinomial NB: {accuracy:.2f}")



In [14]:
print_all_accuracies("SS2030", df_ss2030)
print_all_accuracies("100k Arabic Reviews", df_reviews)

SS2030 dataset accuracy with Multinomial NB: 0.59
100k Arabic Reviews dataset accuracy with Multinomial NB: 0.60


In [15]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')
cv=ShuffleSplit(10,test_size=0.3, random_state=1)
cross_val_score(trained_clf_multinomial_nb,X,y, cv=cv , scoring='accuracy')

array([0.77304965, 0.76638298, 0.77078014, 0.77198582, 0.77375887,
       0.76943262, 0.76950355, 0.77120567, 0.77368794, 0.77425532])

# GridSearchCV

In [16]:
from sklearn.model_selection import GridSearchCV
hyperparameters = {"alpha":(0,0.1, 0.2, 0.3, 0.4, 0.5 ,  0.8 , 0.88, 0.9, 1 ,2, 3,4 ,5, 6)}
Grid= GridSearchCV(estimator=MultinomialNB(), param_grid=hyperparameters, cv=10, scoring="accuracy", n_jobs=-1)
train_model(Grid, X, y)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 GridSearchCV(cv=10, estimator=MultinomialNB(), n_jobs=-1,
                              param_grid={'alpha': (0, 0.1, 0.2, 0.3, 0.4, 0.5,
                                                    0.8, 0.88, 0.9, 1, 2, 3, 4,
                                                    5, 6)},
                              scoring='accuracy'))])

In [17]:
Grid.best_estimator_

MultinomialNB(alpha=0.2)

In [18]:
Grid.best_params_

{'alpha': 0.2}

In [19]:
Grid.best_score_

0.7890212765957447

In [20]:
trained_clf_multinomial_nb_imm = train_model(MultinomialNB(alpha=0.2), X, y)
accuracy = get_accuracy(trained_clf_multinomial_nb_imm,X_test, y_test)
print(f"Test dataset accuracy with MultinomialNB: {accuracy:.2f}")

Test dataset accuracy with MultinomialNB: 0.79


In [21]:
from sklearn.model_selection import GridSearchCV
hyperparameters = {"alpha":(0, 0.8,0.88, 0.9, 1 ,2,3,4,5,6)}
Grid= GridSearchCV(estimator=MultinomialNB(), param_grid=hyperparameters, cv=10, scoring="accuracy", n_jobs=-1)
train_model(Grid, X, y)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 GridSearchCV(cv=10, estimator=MultinomialNB(), n_jobs=-1,
                              param_grid={'alpha': (0, 0.8, 0.88, 0.9, 1, 2, 3,
                                                    4, 5, 6)},
                              scoring='accuracy'))])

In [22]:
Grid.best_estimator_

MultinomialNB(alpha=0.8)

In [23]:
Grid.best_params_

{'alpha': 0.8}

In [24]:
Grid.best_score_

0.7826595744680851

In [25]:
trained_clf_multinomial_nb_im = train_model(MultinomialNB(alpha=0.8), X, y)
accuracy = get_accuracy(trained_clf_multinomial_nb_im,X_test, y_test)
print(f"Test dataset accuracy with MultinomialNB: {accuracy:.2f}")

Test dataset accuracy with MultinomialNB: 0.79


In [26]:
def print_all_im_accuracies(dataset_name, dataset):
  accuracy = get_accuracy(trained_clf_multinomial_nb_im,dataset.tweet.values, dataset.label.values)
  print(f"{dataset_name} dataset accuracy with Multinomial NB: {accuracy:.2f}")

In [27]:
print_all_im_accuracies("SS2030", df_ss2030)
print_all_im_accuracies("100k Arabic Reviews", df_reviews)

SS2030 dataset accuracy with Multinomial NB: 0.59
100k Arabic Reviews dataset accuracy with Multinomial NB: 0.60


In [28]:
from sklearn import  pipeline
nb_model = MultinomialNB()
clf = pipeline.Pipeline([('nb', nb_model)])
param_grid = {'nb__alpha': [0.001, 0.01,0.8, 0.1, 1, 10, 100]}
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring="accuracy",
                                 verbose=10, n_jobs=-1, refit=True, cv=2)

# Fit Grid Search Model
train_model(model, X, y) # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 7 candidates, totalling 14 fits
Best score: 0.771
Best parameters set:
	nb__alpha: 0.1


# ensemble learning

# StackingClassifier

In [29]:
from sklearn.ensemble import  StackingClassifier
model1= StackingClassifier([('MNB1',MultinomialNB(alpha=0.8)),('MNB2',MultinomialNB(alpha=0.2)),
                         ('MNB3',MultinomialNB(alpha=1))], final_estimator=MultinomialNB(alpha=1))



trained_clf_multinomial_nb_im2= train_model(model1, X, y)

In [30]:
accuracy = get_accuracy(trained_clf_multinomial_nb_im2 ,X_test, y_test)
print(f"Test dataset accuracy with MultinomialNB: {accuracy:.2f}")

Test dataset accuracy with MultinomialNB: 0.51


# BaggingClassifier 

In [31]:
from sklearn.ensemble import BaggingClassifier 
modell =BaggingClassifier(base_estimator=MultinomialNB(alpha=1), n_estimators=200)
trained_clf_multinomial_nb_im3= train_model(modell, X, y)

In [32]:
accuracy = get_accuracy(trained_clf_multinomial_nb_im3 ,X_test, y_test)
print(f"Test dataset accuracy with MultinomialNB: {accuracy:.2f}")

Test dataset accuracy with MultinomialNB: 0.78


In [33]:
def print_all_im3_accuracies(dataset_name, dataset):
  accuracy = get_accuracy(trained_clf_multinomial_nb_im3,dataset.tweet.values, dataset.label.values)
  print(f"{dataset_name} dataset accuracy with Multinomial NB: {accuracy:.2f}")
print_all_im3_accuracies("SS2030", df_ss2030)
print_all_im3_accuracies("100k Arabic Reviews", df_reviews)

SS2030 dataset accuracy with Multinomial NB: 0.59
100k Arabic Reviews dataset accuracy with Multinomial NB: 0.60


# VotingClassifier

In [34]:
from sklearn.ensemble import VotingClassifier
model1=MultinomialNB(alpha=1)
model2=MultinomialNB(alpha=1)
model21=MultinomialNB(alpha=1)
model3= VotingClassifier([('mnb1',model1),
                         ('mnb2',model2),('mnb3',model21)
                         ],voting='hard')
trained_clf_multinomial_nb_im41= train_model(model3, X, y)
for model in (model1,model2,model21, model3):
    
    trained_clf_multinomial_nb_im4= train_model(model, X, y)
    accuracy = get_accuracy(trained_clf_multinomial_nb_im4 ,X_test, y_test)
    print(f"Test dataset accuracy with MultinomialNB: {accuracy:.2f}")

Test dataset accuracy with MultinomialNB: 0.79
Test dataset accuracy with MultinomialNB: 0.79
Test dataset accuracy with MultinomialNB: 0.79
Test dataset accuracy with MultinomialNB: 0.79


In [35]:
def print_all_im4_accuracies(dataset_name, dataset):
  accuracy = get_accuracy(trained_clf_multinomial_nb_im41,dataset.tweet.values, dataset.label.values)
  print(f"{dataset_name} dataset accuracy with Multinomial NB: {accuracy:.2f}")
print_all_im4_accuracies("SS2030", df_ss2030)
print_all_im4_accuracies("100k Arabic Reviews", df_reviews)

SS2030 dataset accuracy with Multinomial NB: 0.59
100k Arabic Reviews dataset accuracy with Multinomial NB: 0.60
